# 🌐 APIs REST y Web Scraping para Ingeniería de Datos

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. ✅ Consumir **APIs REST** con `requests`
2. ✅ Manejar **autenticación** y headers
3. ✅ Implementar **paginación** y rate limiting
4. ✅ Realizar **web scraping** con `BeautifulSoup`
5. ✅ Extraer datos de HTML estructurado
6. ✅ Aplicar **mejores prácticas** y ética en scraping
7. ✅ Construir un **pipeline de ingesta** desde APIs

---

## 1. Introducción a APIs REST

### 🤔 ¿Qué es una API REST?

**API REST** (Representational State Transfer) es un estilo de arquitectura para servicios web que usa HTTP.

### Métodos HTTP Comunes:

| Método | Propósito | Ejemplo |
|--------|-----------|----------|
| **GET** | Obtener datos | `/api/usuarios` |
| **POST** | Crear recurso | `/api/usuarios` (con body) |
| **PUT** | Actualizar completo | `/api/usuarios/123` |
| **PATCH** | Actualizar parcial | `/api/usuarios/123` |
| **DELETE** | Eliminar recurso | `/api/usuarios/123` |

### Códigos de Estado HTTP:

- **2xx**: Éxito (200 OK, 201 Created)
- **4xx**: Error del cliente (400 Bad Request, 404 Not Found, 401 Unauthorized)
- **5xx**: Error del servidor (500 Internal Server Error, 503 Service Unavailable)

---

## 2. Configuración Inicial

In [ ]:
# Importaciones necesarias
import requests
import json
import pandas as pd
import time
from typing import List, Dict, Optional
from datetime import datetime
import warnings

# Para web scraping
from bs4 import BeautifulSoup
import re

warnings.filterwarnings('ignore')

print("✅ Librerías cargadas correctamente")
print(f"📦 Requests versión: {requests.__version__}")

### 📚 Bibliotecas para APIs y Web Scraping

**Librerías principales:**
- **requests:** Cliente HTTP para consumir APIs REST
- **json:** Parsing y manipulación de datos JSON
- **BeautifulSoup:** Parser HTML/XML para web scraping
- **pandas:** Estructurar datos extraídos en DataFrames

**Uso en ingeniería de datos:**
- Ingesta desde APIs externas (redes sociales, servicios cloud)
- Extracción de datos de sitios web
- Integración con fuentes de datos sin APIs formales

**Instalación:** `pip install requests beautifulsoup4 lxml`

## 3. Consumir APIs REST Públicas

### 3.1 Ejemplo Básico: JSONPlaceholder

In [ ]:
# API pública para testing
url = "https://jsonplaceholder.typicode.com/posts"

# Realizar petición GET
response = requests.get(url)

# Verificar código de estado
print(f"📡 Status Code: {response.status_code}")
print(f"✅ Exitoso: {response.ok}")

# Obtener datos JSON
if response.ok:
    posts = response.json()
    print(f"\n📊 Total de posts: {len(posts)}")
    print(f"\n🔍 Primer post:")
    print(json.dumps(posts[0], indent=2))

### 🌐 Petición GET Básica con Requests

**Concepto:** `requests.get()` envía solicitud HTTP GET para obtener datos de una API.

**Flujo:**
1. **Enviar petición:** `response = requests.get(url)`
2. **Verificar estado:** `response.status_code` (200 = éxito)
3. **Parsear JSON:** `response.json()` convierte a diccionario Python
4. **Validar:** `response.ok` = True si status 200-299

**Propiedades útiles:**
- `response.text`: contenido como string
- `response.json()`: parsea JSON automáticamente
- `response.headers`: headers de respuesta

**Buena práctica:** Siempre verificar `response.ok` antes de procesar datos.

### 3.2 Convertir Respuesta a DataFrame

In [ ]:
# Convertir a DataFrame
df_posts = pd.DataFrame(posts)

print(f"📊 Shape: {df_posts.shape}")
print(f"\n🔍 Vista previa:")
display(df_posts.head())

# Información del DataFrame
print("\n📋 Info:")
df_posts.info()

### 📊 JSON a DataFrame: Estructurando Datos de API

**Concepto:** `pd.DataFrame()` convierte lista de diccionarios JSON en tabla estructurada.

**Ventajas:**
- Análisis con pandas (filtros, agregaciones)
- Exportación a CSV, SQL, Parquet
- Integración con pipelines ETL

**Conversión automática:**
- Claves JSON → columnas DataFrame
- Lista de objetos → filas
- Tipos inferidos automáticamente

**Uso típico:** Primera transformación tras ingestar datos desde API.

### 3.3 Parámetros de Consulta (Query Parameters)

In [ ]:
# Filtrar por userId
params = {'userId': 1}
response = requests.get(url, params=params)

if response.ok:
    posts_user1 = response.json()
    print(f"📊 Posts del usuario 1: {len(posts_user1)}")
    print(f"\n🔗 URL completa: {response.url}")
    
    # Ver primeros 3
    for post in posts_user1[:3]:
        print(f"\n📝 Post {post['id']}: {post['title'][:50]}...")

### 🔍 Query Parameters: Filtrado en APIs

**Concepto:** Los query parameters permiten filtrar, paginar y personalizar respuestas de APIs.

**Sintaxis:**
```python
params = {'userId': 1, 'limit': 10}
requests.get(url, params=params)
```

**URL generada:** `https://api.com/posts?userId=1&limit=10`

**Casos comunes:**
- Filtros: `?category=tech&status=active`
- Paginación: `?page=2&per_page=50`
- Ordenamiento: `?sort=date&order=desc`

**Ventaja:** Reduce transferencia de datos, obtiene solo lo necesario.

### 3.4 Headers Personalizados

In [ ]:
# Definir headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Data Pipeline v1.0)',
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

response = requests.get(url, headers=headers)

print(f"✅ Status: {response.status_code}")
print(f"\n📄 Headers de la respuesta:")
for key, value in list(response.headers.items())[:5]:
    print(f"  {key}: {value}")

### 📄 Headers HTTP: Metadata de Peticiones

**Concepto:** Headers envían información adicional sobre la petición (autenticación, formato, identidad).

**Headers importantes:**
- **User-Agent:** Identifica tu aplicación/script
- **Accept:** Formato de respuesta deseado (application/json)
- **Authorization:** Token de autenticación (Bearer, API Key)
- **Content-Type:** Tipo de datos enviados (POST/PUT)

**Ejemplo autenticación:**
```python
headers = {'Authorization': 'Bearer YOUR_TOKEN'}
```

**Importancia:** Muchas APIs requieren User-Agent válido o bloquean peticiones sin headers.

## 4. Trabajar con APIs Reales

### 4.1 GitHub API - Datos Públicos

In [ ]:
# Obtener información de un repositorio público
repo_url = "https://api.github.com/repos/pandas-dev/pandas"

response = requests.get(repo_url)

if response.ok:
    repo_data = response.json()
    
    print("📦 INFORMACIÓN DEL REPOSITORIO PANDAS")
    print("=" * 50)
    print(f"Nombre: {repo_data['name']}")
    print(f"Descripción: {repo_data['description'][:80]}...")
    print(f"⭐ Stars: {repo_data['stargazers_count']:,}")
    print(f"🍴 Forks: {repo_data['forks_count']:,}")
    print(f"👁️ Watchers: {repo_data['watchers_count']:,}")
    print(f"🐛 Open Issues: {repo_data['open_issues_count']:,}")
    print(f"📅 Creado: {repo_data['created_at']}")
    print(f"🔄 Última actualización: {repo_data['updated_at']}")
    print(f"📝 Lenguaje principal: {repo_data['language']}")
else:
    print(f"❌ Error: {response.status_code}")

### 4.2 Paginación en APIs

In [ ]:
def obtener_todos_los_commits(owner: str, repo: str, max_pages: int = 3) -> List[Dict]:
    """
    Obtiene commits de un repositorio con paginación.
    """
    base_url = f"https://api.github.com/repos/{owner}/{repo}/commits"
    all_commits = []
    
    for page in range(1, max_pages + 1):
        params = {
            'page': page,
            'per_page': 10  # Registros por página
        }
        
        print(f"📄 Obteniendo página {page}...")
        response = requests.get(base_url, params=params)
        
        if response.ok:
            commits = response.json()
            if not commits:  # No hay más páginas
                break
            all_commits.extend(commits)
        else:
            print(f"❌ Error en página {page}: {response.status_code}")
            break
        
        # Respetar rate limits
        time.sleep(1)
    
    return all_commits

# Ejemplo: Obtener commits de un repo pequeño
commits = obtener_todos_los_commits('octocat', 'Hello-World', max_pages=2)

print(f"\n✅ Total commits obtenidos: {len(commits)}")

# Analizar commits
if commits:
    df_commits = pd.DataFrame([
        {
            'sha': c['sha'][:7],
            'author': c['commit']['author']['name'],
            'date': c['commit']['author']['date'],
            'message': c['commit']['message'][:50]
        }
        for c in commits
    ])
    
    print("\n📊 Últimos commits:")
    display(df_commits.head(10))

### 📄 Paginación: Obtener Grandes Volúmenes de Datos

**Concepto:** Las APIs limitan resultados por petición, requiriendo múltiples requests para datasets completos.

**Estrategias comunes:**
1. **Offset/Limit:** `?offset=100&limit=50`
2. **Page-based:** `?page=3&per_page=100`
3. **Cursor-based:** `?cursor=next_token` (más eficiente)

**Implementación:**
- Loop hasta que respuesta esté vacía o alcance máximo
- `time.sleep()` entre requests para respetar rate limits
- Acumular resultados en lista

**Buena práctica:** Implementar límite máximo de páginas para evitar loops infinitos.

### 4.3 Rate Limiting y Manejo de Errores

In [ ]:
def api_request_con_retry(url: str, max_retries: int = 3, delay: float = 2.0) -> Optional[Dict]:
    """
    Realiza petición con reintentos automáticos.
    """
    for attempt in range(1, max_retries + 1):
        try:
            response = requests.get(url, timeout=10)
            
            if response.ok:
                return response.json()
            
            # Rate limit excedido
            if response.status_code == 429:
                retry_after = int(response.headers.get('Retry-After', delay))
                print(f"⏳ Rate limit alcanzado. Esperando {retry_after}s...")
                time.sleep(retry_after)
                continue
            
            # Otros errores
            if response.status_code >= 500:
                print(f"⚠️ Error del servidor ({response.status_code}). Reintento {attempt}/{max_retries}")
                time.sleep(delay)
                continue
            
            # Error definitivo
            print(f"❌ Error {response.status_code}: {response.text[:100]}")
            return None
            
        except requests.exceptions.Timeout:
            print(f"⏱️ Timeout. Reintento {attempt}/{max_retries}")
            time.sleep(delay)
        except requests.exceptions.ConnectionError:
            print(f"🔌 Error de conexión. Reintento {attempt}/{max_retries}")
            time.sleep(delay)
        except Exception as e:
            print(f"❌ Error inesperado: {str(e)}")
            return None
    
    print(f"❌ Máximo de reintentos alcanzado ({max_retries})")
    return None

# Probar función
data = api_request_con_retry("https://api.github.com/users/octocat")

if data:
    print(f"\n✅ Usuario obtenido: {data['login']}")
    print(f"👤 Nombre: {data['name']}")
    print(f"📍 Ubicación: {data.get('location', 'N/A')}")
    print(f"📦 Repositorios públicos: {data['public_repos']}")

### ⚡ Rate Limiting y Reintentos Automáticos

**Concepto:** APIs limitan número de peticiones por tiempo (ej: 100/hora) para prevenir abuso.

**Manejo de rate limits:**
- **Status 429:** Too Many Requests
- **Header `Retry-After`:** segundos a esperar
- **Backoff exponencial:** incrementar delay tras cada fallo

**Errores comunes:**
- **4xx:** Errores del cliente (request inválido) → no reintentar
- **5xx:** Errores del servidor → reintentar con delay
- **Timeout/Connection:** problemas de red → reintentar

**Estrategia robusta:** 3 reintentos con delays incrementales + captura de excepciones.

## 5. Autenticación en APIs

### 5.1 API Key en Headers

In [ ]:
# Ejemplo genérico (NO ejecutar sin API key real)
def api_con_key(api_key: str, endpoint: str) -> Optional[Dict]:
    """
    Ejemplo de autenticación con API Key.
    """
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Accept': 'application/json'
    }
    
    response = requests.get(endpoint, headers=headers)
    
    if response.ok:
        return response.json()
    else:
        print(f"❌ Error: {response.status_code}")
        return None

print("""
💡 MEJORES PRÁCTICAS PARA API KEYS:

1. ❌ NUNCA hardcodear API keys en el código
2. ✅ Usar variables de entorno:
   import os
   api_key = os.getenv('MI_API_KEY')

3. ✅ Archivo de configuración (no versionado):
   import yaml
   with open('config/credentials.yaml') as f:
       creds = yaml.safe_load(f)
   api_key = creds['api_key']

4. ✅ Agregar credentials.yaml a .gitignore
""")

### 5.2 OAuth 2.0 (Conceptual)

In [ ]:
print("""
🔐 FLUJO OAUTH 2.0:

1. Usuario autoriza aplicación
2. API devuelve código de autorización
3. Aplicación intercambia código por access token
4. Usar access token en peticiones

Ejemplo con Google API:

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

creds = Credentials.from_authorized_user_file('token.json')
service = build('sheets', 'v4', credentials=creds)

📚 Librerías útiles:
  • requests-oauthlib
  • authlib
  • httpx (async)
""")

## 6. Web Scraping con BeautifulSoup

### 6.1 Conceptos Básicos

In [ ]:
# Ejemplo con HTML simple
html_content = """
<html>
<head><title>Productos</title></head>
<body>
    <h1>Catálogo de Productos</h1>
    <div class="producto">
        <h2>Laptop</h2>
        <p class="precio">$1200</p>
        <p class="stock">En stock</p>
    </div>
    <div class="producto">
        <h2>Mouse</h2>
        <p class="precio">$25</p>
        <p class="stock">Agotado</p>
    </div>
    <div class="producto">
        <h2>Teclado</h2>
        <p class="precio">$75</p>
        <p class="stock">En stock</p>
    </div>
</body>
</html>
"""

# Parsear HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Extraer título
titulo = soup.title.string
print(f"📄 Título: {titulo}")

# Extraer encabezado
h1 = soup.find('h1').text
print(f"📌 Encabezado: {h1}")

### 🍲 BeautifulSoup: Parser HTML para Scraping

**Concepto:** BeautifulSoup convierte HTML en árbol navegable de objetos Python.

**Parsers disponibles:**
- `html.parser`: incluido en Python (estándar)
- `lxml`: más rápido, requiere instalación
- `html5lib`: más tolerante con HTML mal formado

**Operaciones básicas:**
- `soup.find('tag')`: primer elemento que coincide
- `soup.find_all('tag')`: todos los elementos
- `soup.title.string`: acceso directo a elementos únicos
- `.text`: extraer texto sin tags HTML

**Uso:** Extraer datos estructurados de páginas web sin API.

### 6.2 Buscar Elementos

In [ ]:
# Buscar todos los productos
productos = soup.find_all('div', class_='producto')

print(f"📦 Total productos encontrados: {len(productos)}\n")

# Extraer información de cada producto
datos_productos = []

for producto in productos:
    nombre = producto.find('h2').text
    precio = producto.find('p', class_='precio').text
    stock = producto.find('p', class_='stock').text
    
    datos_productos.append({
        'nombre': nombre,
        'precio': precio,
        'stock': stock
    })
    
    print(f"🛒 {nombre}")
    print(f"   💰 Precio: {precio}")
    print(f"   📦 Stock: {stock}\n")

# Convertir a DataFrame
df_productos = pd.DataFrame(datos_productos)
display(df_productos)

### 🔍 Selectores CSS y Búsqueda de Elementos

**Concepto:** Localizar elementos específicos en HTML usando clases, IDs, tags y atributos.

**Métodos principales:**
- `find('tag')`: primer elemento
- `find_all('tag')`: lista de todos los elementos
- `find('tag', class_='nombre')`: buscar por clase CSS
- `find('tag', id='nombre')`: buscar por ID
- `find('tag', attrs={'data-id': '123'})`: atributos personalizados

**Selectores CSS avanzados:**
```python
soup.select('.producto')  # por clase
soup.select('#main')      # por ID
soup.select('div > p')    # hijos directos
```

**Extracción:** `.text` para contenido, `.get('href')` para atributos.

### 6.3 Selectores CSS

In [ ]:
# Usar selectores CSS (más potente)
precios = soup.select('.producto .precio')

print("💰 Precios encontrados con selector CSS:")
for precio in precios:
    print(f"  {precio.text}")

# Productos en stock
en_stock = [p for p in soup.select('.producto') 
            if 'En stock' in p.select_one('.stock').text]

print(f"\n✅ Productos en stock: {len(en_stock)}")

### 6.4 Scraping de Sitio Web Real (Ejemplo Ético)

In [ ]:
def scrape_quotes() -> pd.DataFrame:
    """
    Scrape de http://quotes.toscrape.com (sitio de prueba).
    """
    url = "http://quotes.toscrape.com/"
    
    # Headers para simular navegador
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    response = requests.get(url, headers=headers)
    
    if not response.ok:
        print(f"❌ Error: {response.status_code}")
        return pd.DataFrame()
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar todas las quotes
    quotes_elements = soup.find_all('div', class_='quote')
    
    quotes_data = []
    
    for quote_el in quotes_elements:
        text = quote_el.find('span', class_='text').text
        author = quote_el.find('small', class_='author').text
        tags = [tag.text for tag in quote_el.find_all('a', class_='tag')]
        
        quotes_data.append({
            'quote': text,
            'author': author,
            'tags': ', '.join(tags)
        })
    
    return pd.DataFrame(quotes_data)

# Ejecutar scraping
df_quotes = scrape_quotes()

if not df_quotes.empty:
    print(f"✅ Quotes extraídas: {len(df_quotes)}\n")
    display(df_quotes.head())
    
    # Análisis
    print("\n📊 Autores más citados:")
    print(df_quotes['author'].value_counts().head())

### 🌐 Scraping de Sitios Reales: Flujo Completo

**Concepto:** Extraer datos de sitios web públicos siguiendo mejores prácticas éticas y técnicas.

**Pasos esenciales:**
1. **Revisar robots.txt:** `sitio.com/robots.txt` - qué está permitido
2. **User-Agent válido:** identificarse como navegador
3. **Parsear HTML:** BeautifulSoup convierte respuesta en objeto navegable
4. **Extraer datos:** usar selectores para localizar información
5. **Estructurar:** convertir a DataFrame para análisis

**Sitios de práctica:**
- quotes.toscrape.com (diseñado para aprender)
- books.toscrape.com (catálogo de libros)

**Ética:** Respetar términos de servicio, no sobrecargar servidores, usar datos responsablemente.

### 6.5 Extraer Tablas HTML

In [ ]:
# HTML con tabla
html_tabla = """
<table>
    <tr>
        <th>País</th>
        <th>Capital</th>
        <th>Población (M)</th>
    </tr>
    <tr>
        <td>México</td>
        <td>Ciudad de México</td>
        <td>126</td>
    </tr>
    <tr>
        <td>Colombia</td>
        <td>Bogotá</td>
        <td>51</td>
    </tr>
    <tr>
        <td>Argentina</td>
        <td>Buenos Aires</td>
        <td>45</td>
    </tr>
</table>
"""

soup_tabla = BeautifulSoup(html_tabla, 'html.parser')
tabla = soup_tabla.find('table')

# Extraer headers
headers = [th.text for th in tabla.find_all('th')]

# Extraer filas
filas = []
for tr in tabla.find_all('tr')[1:]:  # Saltar header
    celdas = [td.text for td in tr.find_all('td')]
    filas.append(celdas)

# Crear DataFrame
df_paises = pd.DataFrame(filas, columns=headers)
df_paises['Población (M)'] = df_paises['Población (M)'].astype(int)

print("🌍 Tabla extraída:")
display(df_paises)

# Alternativa con pandas (más simple)
print("\n💡 Con pandas.read_html():")
df_paises_pandas = pd.read_html(html_tabla)[0]
display(df_paises_pandas)

### 📊 Extracción de Tablas HTML

**Concepto:** Las tablas HTML (`<table>`) son estructuras ideales para convertir directamente a DataFrames.

**Estrategia:**
1. Localizar `<table>` con `soup.find('table')`
2. Extraer headers de `<th>` en primera fila
3. Iterar sobre `<tr>` (filas) extrayendo `<td>` (celdas)
4. Crear DataFrame con headers y filas

**Alternativa rápida:**
```python
pd.read_html(url)  # Pandas detecta tablas automáticamente
```

**Uso común:** Datos estadísticos, rankings, resultados deportivos, precios históricos.

## 7. Pipeline de Ingesta desde API

### 7.1 Clase APIExtractor

In [ ]:
class APIExtractor:
    """
    Extractor genérico para APIs REST con manejo de errores y paginación.
    """
    
    def __init__(self, base_url: str, headers: Optional[Dict] = None, rate_limit: float = 1.0):
        self.base_url = base_url
        self.headers = headers or {}
        self.rate_limit = rate_limit
        self.session = requests.Session()
        self.session.headers.update(self.headers)
    
    def get(self, endpoint: str, params: Optional[Dict] = None) -> Optional[Dict]:
        """Realiza petición GET con manejo de errores."""
        url = f"{self.base_url}/{endpoint}"
        
        try:
            response = self.session.get(url, params=params, timeout=30)
            response.raise_for_status()
            time.sleep(self.rate_limit)  # Rate limiting
            return response.json()
        except requests.exceptions.HTTPError as e:
            print(f"❌ HTTP Error: {e}")
            return None
        except requests.exceptions.Timeout:
            print(f"⏱️ Timeout al consultar {url}")
            return None
        except Exception as e:
            print(f"❌ Error inesperado: {str(e)}")
            return None
    
    def get_paginated(self, endpoint: str, page_param: str = 'page', 
                     max_pages: int = 10) -> List[Dict]:
        """Obtiene datos paginados."""
        all_data = []
        
        for page in range(1, max_pages + 1):
            print(f"📄 Página {page}/{max_pages}")
            params = {page_param: page}
            data = self.get(endpoint, params)
            
            if not data:
                break
            
            # Asumiendo que data es una lista
            if isinstance(data, list):
                if not data:  # Lista vacía
                    break
                all_data.extend(data)
            else:
                # Si es dict, agregar directamente
                all_data.append(data)
        
        return all_data
    
    def to_dataframe(self, data: List[Dict]) -> pd.DataFrame:
        """Convierte datos a DataFrame."""
        return pd.DataFrame(data)

print("✅ Clase APIExtractor definida")

### 🏗️ Pipeline de Ingesta: Clase APIExtractor

**Concepto:** Encapsular lógica de extracción desde APIs en clase reutilizable.

**Componentes clave:**
- **Session:** `requests.Session()` reutiliza conexiones (más eficiente)
- **Rate limiting:** `time.sleep()` entre peticiones
- **Manejo de errores:** try/except para HTTPError, Timeout, etc.
- **Paginación automática:** itera hasta encontrar datos vacíos
- **Conversión:** método `.to_dataframe()` para análisis

**Ventajas:**
- Reutilizable para múltiples APIs
- Código más limpio y mantenible
- Configuración centralizada (headers, rate limit)

**Uso:** Base para pipelines ETL de ingesta desde APIs externas.

### 7.2 Usar APIExtractor

In [ ]:
# Crear extractor para JSONPlaceholder
extractor = APIExtractor(
    base_url="https://jsonplaceholder.typicode.com",
    rate_limit=0.5  # 0.5 segundos entre requests
)

# Extraer usuarios
print("🔍 Extrayendo usuarios...\n")
usuarios = extractor.get('users')

if usuarios:
    df_usuarios = extractor.to_dataframe(usuarios)
    
    # Seleccionar columnas relevantes
    df_usuarios_clean = df_usuarios[['id', 'name', 'email', 'phone', 'website']].copy()
    
    print(f"✅ Usuarios extraídos: {len(df_usuarios_clean)}")
    display(df_usuarios_clean.head())
    
    # Guardar
    output_path = '../../outputs/usuarios_api.csv'
    df_usuarios_clean.to_csv(output_path, index=False)
    print(f"\n💾 Datos guardados en: {output_path}")

## 8. Mejores Prácticas y Ética

### 8.1 Consideraciones Legales y Éticas

In [ ]:
print("""
⚖️ MEJORES PRÁCTICAS EN WEB SCRAPING:

1. 📜 VERIFICAR LEGALIDAD
   • Revisar Términos de Servicio
   • Consultar robots.txt
   • Respetar copyright

2. 🤝 SER RESPETUOSO
   • Usar rate limiting (delays)
   • No sobrecargar servidores
   • Scraping en horarios de bajo tráfico

3. 🔒 PRIVACIDAD
   • No extraer datos personales sensibles
   • Cumplir GDPR/CCPA si aplica
   • Anonimizar datos cuando sea posible

4. 🎯 ALTERNATIVAS PREFERIBLES
   • Usar API oficial si existe
   • Contactar al propietario del sitio
   • Considerar datasets públicos

5. 🛡️ IDENTIFICARSE
   • User-Agent descriptivo
   • Email de contacto
   • Documentar propósito

6. 💾 CACHÉ LOCAL
   • No repetir requests innecesarios
   • Almacenar resultados localmente
   • Actualizar solo cuando sea necesario

❌ NUNCA:
  • Ignorar robots.txt
  • Hacer requests masivos sin delays
  • Usar datos para spam o fraude
  • Redistribuir contenido con copyright
  • Evadir CAPTCHAs o medidas de seguridad
""")

### 8.2 Verificar robots.txt

In [ ]:
def verificar_robots_txt(url: str) -> None:
    """
    Verifica el archivo robots.txt de un sitio.
    """
    from urllib.parse import urlparse
    
    parsed = urlparse(url)
    robots_url = f"{parsed.scheme}://{parsed.netloc}/robots.txt"
    
    try:
        response = requests.get(robots_url, timeout=10)
        if response.ok:
            print(f"🤖 robots.txt de {parsed.netloc}:\n")
            print(response.text[:500])  # Primeros 500 caracteres
        else:
            print(f"⚠️ No se encontró robots.txt (Status: {response.status_code})")
    except Exception as e:
        print(f"❌ Error al verificar robots.txt: {str(e)}")

# Ejemplo
verificar_robots_txt("https://github.com")

## 🎯 Ejercicios Prácticos

### Ejercicio 1: API de GitHub
Crea un script que:
1. Obtenga los últimos 50 repositorios de un usuario de GitHub
2. Extraiga: nombre, descripción, estrellas, lenguaje, fecha de creación
3. Convierta a DataFrame y guarde en CSV
4. Genere estadísticas (lenguaje más usado, repo con más estrellas)

In [ ]:
# TU CÓDIGO AQUÍ

### Ejercicio 2: Web Scraping
Scrape http://books.toscrape.com:
1. Extrae título, precio, disponibilidad y rating
2. Navega múltiples páginas (al menos 3)
3. Limpia los datos (quitar símbolos de precio, convertir ratings)
4. Crea visualización de precio promedio por rating

In [ ]:
# TU CÓDIGO AQUÍ

### Ejercicio 3: Pipeline Completo
Implementa un pipeline que:
1. Extraiga datos de una API pública (elige una de [public-apis.io](https://public-apis.io))
2. Implemente manejo de errores robusto
3. Aplique transformaciones necesarias
4. Guarde resultados en formato Parquet
5. Genere un reporte con estadísticas básicas

In [ ]:
# TU CÓDIGO AQUÍ

## 📚 Recursos Adicionales

### APIs Públicas para Practicar
- [JSONPlaceholder](https://jsonplaceholder.typicode.com/) - API fake para testing
- [GitHub API](https://docs.github.com/en/rest) - Datos de repositorios
- [OpenWeatherMap](https://openweathermap.org/api) - Datos meteorológicos
- [NewsAPI](https://newsapi.org/) - Noticias globales
- [Public APIs](https://github.com/public-apis/public-apis) - Lista curada

### Sitios para Practicar Scraping (Legalmente)
- [Quotes to Scrape](http://quotes.toscrape.com/)
- [Books to Scrape](http://books.toscrape.com/)
- [Scrape This Site](https://www.scrapethissite.com/)

### Documentación
- [Requests Documentation](https://requests.readthedocs.io/)
- [BeautifulSoup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Scrapy](https://scrapy.org/) - Framework avanzado de scraping

### Alternativas Avanzadas
- **Selenium**: Para sitios con JavaScript
- **Playwright**: Automatización de navegadores
- **httpx**: Cliente HTTP async
- **aiohttp**: Requests asíncronos

---

## ✅ Resumen

En este notebook aprendiste:

1. ✅ **APIs REST**: Métodos HTTP, códigos de estado, requests
2. ✅ **Biblioteca requests**: GET, POST, headers, parámetros
3. ✅ **Paginación**: Obtener datos en múltiples páginas
4. ✅ **Rate limiting**: Respetar límites de APIs
5. ✅ **Autenticación**: API keys, OAuth
6. ✅ **BeautifulSoup**: Parsear HTML, selectores CSS
7. ✅ **Web scraping ético**: Robots.txt, mejores prácticas
8. ✅ **Pipeline de ingesta**: Clase reutilizable para APIs

**🎯 Próximo paso**: Proyecto Integrador 1

---

---## 🧭 Navegación**← Anterior:** [← Git y Control de Versiones](07_git_control_versiones.ipynb)**Siguiente →:** [Proyecto Integrador 1 →](09_proyecto_integrador_1.ipynb)**📚 Índice de Nivel Junior:**- [Introducción a la Ingeniería de Datos](01_introduccion_ingenieria_datos.ipynb)- [Programación en Python para Datos](02_python_manipulacion_datos.ipynb)- [Pandas Fundamentos](03_pandas_fundamentos.ipynb)- [SQL Básico](04_sql_basico.ipynb)- [Limpieza de Datos](05_limpieza_datos.ipynb)- [Visualización de Datos](06_visualizacion_datos.ipynb)- [Git y Control de Versiones](07_git_control_versiones.ipynb)- [APIs y Web Scraping](08_apis_web_scraping.ipynb) ← 🔵 Estás aquí- [Proyecto Integrador 1](09_proyecto_integrador_1.ipynb)- [Proyecto Integrador 2](10_proyecto_integrador_2.ipynb)**🎓 Otros Niveles:**- [Nivel Junior](../nivel_junior/README.md)- [Nivel Mid](../nivel_mid/README.md)- [Nivel Senior](../nivel_senior/README.md)- [Nivel GenAI](../nivel_genai/README.md)- [Negocio LATAM](../negocios_latam/README.md)

---## 🧭 Navegación**← Anterior:** [← Git y Control de Versiones](07_git_control_versiones.ipynb)**Siguiente →:** [Proyecto Integrador 1 →](09_proyecto_integrador_1.ipynb)**📚 Índice de Nivel Junior:**- [Introducción a la Ingeniería de Datos](01_introduccion_ingenieria_datos.ipynb)- [Programación en Python para Datos](02_python_manipulacion_datos.ipynb)- [Pandas Fundamentos](03_pandas_fundamentos.ipynb)- [SQL Básico](04_sql_basico.ipynb)- [Limpieza de Datos](05_limpieza_datos.ipynb)- [Visualización de Datos](06_visualizacion_datos.ipynb)- [Git y Control de Versiones](07_git_control_versiones.ipynb)- [APIs y Web Scraping](08_apis_web_scraping.ipynb) ← 🔵 Estás aquí- [Proyecto Integrador 1](09_proyecto_integrador_1.ipynb)- [Proyecto Integrador 2](10_proyecto_integrador_2.ipynb)**🎓 Otros Niveles:**- [Nivel Junior](../nivel_junior/README.md)- [Nivel Mid](../nivel_mid/README.md)- [Nivel Senior](../nivel_senior/README.md)- [Nivel GenAI](../nivel_genai/README.md)- [Negocio LATAM](../negocios_latam/README.md)

---## 🧭 Navegación**← Anterior:** [← Git y Control de Versiones](07_git_control_versiones.ipynb)**Siguiente →:** [Proyecto Integrador 1 →](09_proyecto_integrador_1.ipynb)**📚 Índice de Nivel Junior:**- [Introducción a la Ingeniería de Datos](01_introduccion_ingenieria_datos.ipynb)- [Programación en Python para Datos](02_python_manipulacion_datos.ipynb)- [Pandas Fundamentos](03_pandas_fundamentos.ipynb)- [SQL Básico](04_sql_basico.ipynb)- [Limpieza de Datos](05_limpieza_datos.ipynb)- [Visualización de Datos](06_visualizacion_datos.ipynb)- [Git y Control de Versiones](07_git_control_versiones.ipynb)- [APIs y Web Scraping](08_apis_web_scraping.ipynb) ← 🔵 Estás aquí- [Proyecto Integrador 1](09_proyecto_integrador_1.ipynb)- [Proyecto Integrador 2](10_proyecto_integrador_2.ipynb)**🎓 Otros Niveles:**- [Nivel Junior](../nivel_junior/README.md)- [Nivel Mid](../nivel_mid/README.md)- [Nivel Senior](../nivel_senior/README.md)- [Nivel GenAI](../nivel_genai/README.md)- [Negocio LATAM](../negocios_latam/README.md)